In [1]:
import torch
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
import transformers
from torch.utils.data import Dataset, DataLoader
#from transformers import *
import torch.optim as optim
# from tqdm import tqdm
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import scipy
from scipy.stats import pearsonr
from torch.optim.lr_scheduler import StepLR

In [2]:
!python -m pip install wandb

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

In [4]:
import wandb

# Set your wandb API key
wandb_api_key = "e62fc492915628e64fcac9c082089ffed84dc72d"

# Log in to wandb using the API key
wandb.login(key=wandb_api_key)

# Initialize wandb
wandb.init(project="nlp3-1a")

# Now you can use wandb for logging during training

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240411_103628-n6s4dc68
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wandering-jazz-115
wandb: ⭐️ View project at https://wandb.ai/karan912/nlp3-1a
wandb: 🚀 View run at https://wandb.ai/karan912/nlp3-1a/runs/n6s4dc68/workspace


In [5]:
torch.manual_seed(0)
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

In [7]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data=data
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data.iloc[idx, 0]
        y = self.data.iloc[idx, 1]
        encoded_x = self.tokenizer(x,
                                      add_special_tokens=True, 
                                      padding='max_length', 
                                      truncation=True, 
                                      return_tensors='pt')
        input_ids = encoded_x['input_ids'].squeeze(0)
        attention_mask = encoded_x['attention_mask'].squeeze(0)
        return input_ids,attention_mask,torch.tensor(y)

In [9]:
x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [10]:
BATCH_SIZE=16
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'


In [11]:
def dataloaderbuilder(dataset,batch_size):
    loader=DataLoader(dataset,batch_size=batch_size,shuffle=False)
    return loader

In [12]:
train_dataframe=pd.DataFrame({'text':x_train,'label':y_train})
test_dataframe=pd.DataFrame({'text':x_test,'label':y_test})
val_dataframe=pd.DataFrame({'text':x_val,'label':y_val})

In [13]:
train_data=CustomDataset(train_dataframe,tokenizer)
val_data=CustomDataset(val_dataframe,tokenizer)
test_data=CustomDataset(test_dataframe,tokenizer)

In [14]:
print(len(train_data),len(val_data),len(test_data))

3200 800 1000


In [15]:
train_dataloader=dataloaderbuilder(train_data,BATCH_SIZE)
val_dataloader=dataloaderbuilder(val_data,BATCH_SIZE)
test_dataloader=dataloaderbuilder(test_data,BATCH_SIZE)

In [16]:
print(len(train_dataloader),len(val_dataloader),len(test_dataloader))

200 50 63


In [17]:
torch.cuda.empty_cache()

In [18]:
def plotgraph(train_losses,val_losses):
    x=[i+1 for i in range(NUM_EPOCHS)]
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.legend()
    plt.savefig('loss_graph.png')
    plt.show()

In [19]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [20]:
from torch import nn
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3, hidden_dropout_prob=0.07, attention_probs_dropout_prob=0.07)
# model.classifier = nn.Sequential(
#     nn.Linear(model.config.hidden_size, 1),
#     nn.Sigmoid()  # Output float between 0 and 1
# )
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
# optimizer=torch.optim.SGD(model.parameters(),lr=0.0005,weight_decay=0.001)
loss_fn=nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
print(len(train_dataloader))

200


In [23]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

In [24]:
import gc
import sys
gc.collect()
torch.cuda.empty_cache()

train_losses=[]
val_losses=[]
def train_epoch(model, optimizer,epoch):
    model.train()
    losses = 0
    progress = tqdm(train_dataloader, desc=f"Epoch:{epoch}",total=len(train_dataloader), leave=False)
    i=0
    for batch in progress:
        input_ids = batch[0].to(DEVICE)
        attention_mask = batch[1].to(DEVICE)
        labels = batch[2].to(DEVICE)
        optimizer.zero_grad()
        logits=model(input_ids,attention_mask=attention_mask,labels=labels)
#         loss=loss_fn(logits,labels)
        loss=logits.loss
        losses += loss.item()
#         print(i,loss.item())
        loss.backward()
        optimizer.step()
        del input_ids
        del attention_mask
        del labels
        del logits
        gc.collect()
        torch.cuda.empty_cache()
        progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})
        
    
    x = losses / len(list(train_dataloader))
    train_losses.append(x)
    wandb.log({'epoch':epoch,'train_loss':x})
    tqdm.write(f"Epoch:{epoch}, Avg Train Loss: {x}")
    gc.collect()
    torch.cuda.empty_cache()
    return x



def evaluate(model):
    with torch.no_grad():
        model.eval()
        losses = 0
        batch_pred=[]
        batch_truth=[]
        for batch in val_dataloader:
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            labels = batch[2].to(DEVICE)

            logits=model(input_ids,attention_mask=attention_mask).logits.to(torch.float64)
            batch_pred.append(logits)
            batch_truth.append(labels)

            loss=loss_fn(logits,labels)
            losses += loss.item()
            del input_ids
            del attention_mask
            del labels
            del logits
            gc.collect()
            torch.cuda.empty_cache()

        x = losses / len(list(val_dataloader))
        val_losses.append(x)
        wandb.log({'epoch':epoch,'val_loss':x})
        gc.collect()
        torch.cuda.empty_cache()
        return x

In [25]:
for epoch in range(1, NUM_EPOCHS+1):
    train_loss = train_epoch(model, optimizer,epoch)
    val_loss = evaluate(model)
    scheduler.step(val_loss)
    print("Current LR:", optimizer.param_groups[0]['lr'])
    torch.save(model.state_dict(), f"model1A_epoch_{epoch}.pth")
    wandb.log({'epoch':epoch,'train_loss':train_loss,'val_loss':val_loss})
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}"))


Epoch:1:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch:1, Avg Train Loss: 1.0398657113313674
Current LR: 0.0001
Epoch: 1, Train loss: 1.040, Val loss: 1.021


Epoch:2:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch:2, Avg Train Loss: 1.041785514354706
Current LR: 0.0001
Epoch: 2, Train loss: 1.042, Val loss: 1.052


Epoch:3:   0%|          | 0/200 [00:00<?, ?it/s]

...

Epoch:8, Avg Train Loss: 1.0359962028265
Current LR: 1e-05
Epoch: 8, Train loss: 1.036, Val loss: 1.039


Epoch:9:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch:9, Avg Train Loss: 1.0349878761172295
Epoch 00009: reducing learning rate of group 0 to 1.0000e-06.
Current LR: 1.0000000000000002e-06
Epoch: 9, Train loss: 1.035, Val loss: 1.039


Epoch:10:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch:10, Avg Train Loss: 1.0341926693916321
Current LR: 1.0000000000000002e-06
Epoch: 10, Train loss: 1.034, Val loss: 1.039


In [26]:
wandb.finish()
torch.cuda.empty_cache()


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
wandb: train_loss ▅▅▆▆██▆▆▆▆▂▂▃▃▂▂▁▁▁▁
wandb:   val_loss ▁▁██▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅
wandb: 
wandb: Run summary:
wandb:      epoch 10
wandb: train_loss 1.03419
wandb:   val_loss 1.039
wandb: 
wandb: 🚀 View run wandering-jazz-115 at: https://wandb.ai/karan912/nlp3-1a/runs/n6s4dc68/workspace
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240411_103628-n6s4dc68/logs


In [27]:
#test
def evaluate(model,val_dataloader):
    with torch.no_grad():
        model.eval()
        losses = 0
        progress = tqdm(val_dataloader, desc=f"Epoch:{epoch}",total=len(val_dataloader), leave=False)
        total_logits=[]
        for batch in progress:
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            logits=model(input_ids,attention_mask=attention_mask).logits.to(torch.float64)
            _, predicted = torch.max(logits, 1)
            total_logits.extend(list(predicted.cpu().detach().numpy()))
            
        return total_logits

In [28]:
test_pred = evaluate(model,test_dataloader)
print(test_pred)

Epoch:10:   0%|          | 0/63 [00:00<?, ?it/s]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [29]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [30]:
y_true=y_test

In [31]:
print(classification_report(y_true, test_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       174
           1       0.00      0.00      0.00       403
           2       0.42      1.00      0.59       423

    accuracy                           0.42      1000
   macro avg       0.14      0.33      0.20      1000
weighted avg       0.18      0.42      0.25      1000



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
